In [33]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pygrib
from scipy.interpolate import interp2d
from sklearn import linear_model
clf = linear_model.LinearRegression()
from scipy.stats import pearsonr

In [3]:
ds = xr.open_dataset('./alaska_data/EOF/prcpma_monthly_accum.nc')
prcp = ds.prcp.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/tempma_monthly_mean.nc')
temp = ds.temp.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/vpd_monthly_mean.nc')
vpd = ds.vpd.values
ds.close()

In [3]:
grib = pygrib.open('./alaska_data/temp_2001.grib')
[lat, lon] = grib.message(1).latlons()
flag = grib.message(1).values.mask

In [42]:
fm = interp2d(lon1, lat1, flag[17:-14,10:], kind='cubic')
mask_ip = fm(gridx, gridy)
mask_ip = np.where(mask_ip > 0.5, 1, 0)

lat1 = lat[17:-14,0]
lon1 = lon[0,10:]

In [53]:
gridx = np.arange(-168, -139.99, 0.01)
gridy = np.arange(71.3, 58.39, -0.01)

In [108]:
t = 156
x = 1291
y = 2801
temp_ip = np.zeros((t, x, y))
prcp_ip = np.zeros((t, x, y))
vpd_ip = np.zeros((t, x, y))
for i in range(12, 168):
    data1 = temp[i,17:-14,10:]
    data1 = np.nan_to_num(data1,nan=np.nanmean(data1))
    f1 = interp2d(lon1, lat1, data1, kind='cubic')
    temp_ip[i-12,:,:] = f1(gridx, gridy)
    temp_ip[i-12,:,:] = np.where(mask_ip != 1, temp_ip[i-12,:,:], np.nan)
    
    data2 = prcp[i,17:-14,10:]
    data2 = np.nan_to_num(data2,nan=np.nanmean(data2))
    f2 = interp2d(lon1, lat1, data2, kind='cubic')
    prcp_ip[i-12,:,:] = f2(gridx, gridy)
    prcp_ip[i-12,:,:] = np.where(mask_ip != 1, prcp_ip[i-12,:,:], np.nan)
    
    data3 = vpd[i,17:-14,10:]
    data3 = np.nan_to_num(data3,nan=np.nanmean(data3))
    f3 = interp2d(lon1, lat1, data3, kind='cubic')
    vpd_ip[i-12,:,:] = f3(gridx, gridy)
    vpd_ip[i-12,:,:] = np.where(mask_ip != 1, vpd_ip[i-12,:,:], np.nan)

In [128]:
Lat = np.zeros((t, x, y))
Lon = np.zeros((t, x, y))
for i in range(t):
    Lon[i,:,:], Lat[i,:,:] = np.meshgrid(gridx, gridy[::-1])

In [129]:
out1 = xr.Dataset({ "temp": (("time","south_north", "west_east"), temp_ip)}, 
                          coords={"lat": (("time", "south_north", "west_east"), Lat), 
                                  "lon": (("time", "south_north", "west_east"), Lon)})
out1.to_netcdf('./alaska_data/EOF/temp_interp.nc')

out2 = xr.Dataset({ "prcp": (("time","south_north", "west_east"), prcp_ip)}, 
                          coords={"lat": (("time", "south_north", "west_east"), Lat), 
                                  "lon": (("time", "south_north", "west_east"), Lon)})
out2.to_netcdf('./alaska_data/EOF/prcp_interp.nc')

out3 = xr.Dataset({ "vpd": (("time","south_north", "west_east"), vpd_ip)}, 
                          coords={"lat": (("time", "south_north", "west_east"), Lat), 
                                  "lon": (("time", "south_north", "west_east"), Lon)})
out3.to_netcdf('./alaska_data/EOF/vpd_interp.nc')

In [7]:
# plt.contourf(lon,lat,vpd[7,:,:])
# plt.colorbar()

In [51]:
# ds = xr.open_dataset('./alaska_data/EOF/NIR_interp.nc')
# nir0 = ds.NIR_TS.values[7,:,::-1].T
# mask = np.where(nir0!=nir0[0,0], False, True)
# mask_ip = np.where(mask_ip == 0 , False, True)
# mm = np.logical_or(mask,mask_ip)
# plt.contourf(mm)
# mmm = np.where(mm, np.nan, 1)
# plt.contourf(mmm)
# a = xr.DataArray(mm)
# a.name = 'mask'
# a.to_netcdf('./alaska_data/EOF/mask.nc')

In [7]:
ds = xr.open_dataset('./alaska_data/EOF/temp_interp.nc')
temp = ds.temp.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/prcp_interp.nc')
prcp = ds.prcp.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/vpd_interp.nc')
vpd = ds.vpd.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/NIR_Interp.nc')
nir = ds.NIRvTS.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/mask.nc')
mm = ds.mask.values
ds.close()

In [52]:
t = 156
x = 1291
y = 2801
temp_ip = np.zeros((t, x, y))
prcp_ip = np.zeros((t, x, y))
vpd_ip = np.zeros((t, x, y))
for i in range(t):
    temp_ip[i,:,:] = np.where(mm, np.nan, temp[i,:,:])
    prcp_ip[i,:,:] = np.where(mm, np.nan, prcp[i,:,:])
    vpd_ip[i,:,:] = np.where(mm, np.nan, vpd[i,:,:])

In [8]:
t = 156
x = 1291
y = 2801
nir_ip = np.zeros((t, x, y))
for i in range(t):
    nir_ip[i,:,:] = np.where(mm, np.nan, nir[i,:,::-1].T)

In [9]:
x = np.arange(-168, -139.99, 0.01)
y = np.arange(58.39, 71.3, 0.01)
[Lon,Lat] = np.meshgrid(x,y)
out1 = xr.Dataset({ "temp": (("time","south_north", "west_east"), temp_ip)}, 
                          coords={"lat": (("south_north", "west_east"), Lat), 
                                  "lon": (("south_north", "west_east"), Lon)})
out1.to_netcdf('./alaska_data/EOF/temp_interp.nc')

out2 = xr.Dataset({ "prcp": (("time","south_north", "west_east"), prcp_ip)}, 
                          coords={"lat": (("south_north", "west_east"), Lat), 
                                  "lon": (("south_north", "west_east"), Lon)})
out2.to_netcdf('./alaska_data/EOF/prcp_interp.nc')

out3 = xr.Dataset({ "vpd": (("time","south_north", "west_east"), vpd_ip)}, 
                          coords={"lat": (("south_north", "west_east"), Lat), 
                                  "lon": (("south_north", "west_east"), Lon)})
out3.to_netcdf('./alaska_data/EOF/vpd_interp.nc')

out4 = xr.Dataset({ "nir": (("time","south_north", "west_east"), nir_ip)}, 
                          coords={"lat": (("south_north", "west_east"), Lat), 
                                  "lon": (("south_north", "west_east"), Lon)})
out4.to_netcdf('./alaska_data/EOF/nir_interp.nc')

In [7]:
ds = xr.open_dataset('./alaska_data/EOF/temp_interp.nc')
temp = ds.temp.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/prcp_interp.nc')
prcp = ds.prcp.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/vpd_interp.nc')
vpd = ds.vpd.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/temp_interp.nc')
lat = ds.lat.values
lon = ds.lon.values
ds.close()

In [42]:
t = 156
x = 1291
y = 2801
temp_tr = np.zeros((12,x,y))
prcp_tr = np.zeros((12,x,y))
vpd_tr = np.zeros((12,x,y))
for i in range(12):
    temp_tr[i,:,:] = temp[i::12,:,:].mean(axis=0)
    prcp_tr[i,:,:] = prcp[i::12,:,:].mean(axis=0)
    vpd_tr[i,:,:] = vpd[i::12,:,:].mean(axis=0)

In [47]:
temp_ip_detr = np.zeros((t, x, y))
prcp_ip_detr = np.zeros((t, x, y))
vpd_ip_detr = np.zeros((t, x, y))
for i in range(12):
    for j in range(13):
        temp_ip_detr[int(i+12*j),:,:] = temp[int(i+12*j),:,:] - temp_tr[i,:,:]
        prcp_ip_detr[int(i+12*j),:,:] = prcp[int(i+12*j),:,:] - prcp_tr[i,:,:]
        vpd_ip_detr[int(i+12*j),:,:] = vpd[int(i+12*j),:,:] - vpd_tr[i,:,:]

In [49]:
x = np.arange(-168, -139.99, 0.01)
y = np.arange(58.39, 71.3, 0.01)
[Lon,Lat] = np.meshgrid(x,y)
out1 = xr.Dataset({ "temp": (("time","south_north", "west_east"), temp_ip_detr)}, 
                          coords={"lat": (("south_north", "west_east"), Lat), 
                                  "lon": (("south_north", "west_east"), Lon)})
out1.to_netcdf('./alaska_data/EOF/temp_detr.nc')

out2 = xr.Dataset({ "prcp": (("time","south_north", "west_east"), prcp_ip_detr)}, 
                          coords={"lat": (("south_north", "west_east"), Lat), 
                                  "lon": (("south_north", "west_east"), Lon)})
out2.to_netcdf('./alaska_data/EOF/prcp_detr.nc')

out3 = xr.Dataset({ "vpd": (("time","south_north", "west_east"), vpd_ip_detr)}, 
                          coords={"lat": (("south_north", "west_east"), Lat), 
                                  "lon": (("south_north", "west_east"), Lon)})
out3.to_netcdf('./alaska_data/EOF/vpd_detr.nc')


In [59]:
ds = xr.open_dataset('./alaska_data/EOF/temp_interp.nc')
temp = ds.temp.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/prcp_interp.nc')
prcp = ds.prcp.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/vpd_interp.nc')
vpd = ds.vpd.values
ds.close()
ds = xr.open_dataset('./alaska_data/EOF/temp_interp.nc')
lat = ds.lat.values
lon = ds.lon.values
ds.close()

In [60]:
x = 1291
y = 2801
temp_gm = np.zeros((x,y))
vpd_gm = np.zeros((x,y))
tt = np.zeros((13,x,y))
vv = np.zeros((13,x,y))
for i in range(13):
    tt[i,:,:] = temp[(5+i*12):(8+i*12),:,:].mean(axis=0)
    vv[i,:,:] = vpd[(5+i*12):(8+i*12),:,:].mean(axis=0)
    
temp_gm = tt.mean(axis=0)
vpd_gm = vv.mean(axis=0)

# x = 1291
# y = 2801
# nir_gm = np.zeros((x,y))
# nn = np.zeros((13,x,y))
# for i in range(13):
#     nn[i,:,:] = nir[(5+i*12):(8+i*12),:,:].max(axis=0)
    
# nir_gm = nn.mean(axis=0)

In [25]:
prcp_ya = np.zeros((x,y))
prcp_ya = (prcp[8:(8+12*12),:,:].sum(axis=0))/12

In [28]:
x = np.arange(-168, -139.99, 0.01)
y = np.arange(58.39, 71.3, 0.01)
[Lon,Lat] = np.meshgrid(x,y)
out = xr.Dataset({ "temp": (("south_north", "west_east"), temp_gm),
                  "vpd": (("south_north", "west_east"), vpd_gm),
                  "prcp": (("south_north", "west_east"), 1000*prcp_ya)}, 
                          coords={"lat": (("south_north", "west_east"), Lat), 
                                  "lon": (("south_north", "west_east"), Lon)})
out.to_netcdf('./alaska_data/EOF/olsdata.nc')

In [52]:
x = 1291
y = 2801

temp_tt = np.reshape(temp_gm, (x*y), order='F')
temp_tt = np.ma.masked_array(temp_tt, np.isnan(temp_tt))
ocean = temp_tt.mask
land = ~ocean
temp_tt = temp_tt[land]

prcp_tt = np.reshape(prcp_ya, (x*y), order='F')
prcp_tt = np.ma.masked_array(prcp_tt, np.isnan(prcp_tt))
ocean = prcp_tt.mask
land = ~ocean
prcp_tt = prcp_tt[land]

vpd_tt = np.reshape(vpd_gm, (x*y), order='F')
vpd_tt = np.ma.masked_array(vpd_tt, np.isnan(vpd_tt))
ocean = vpd_tt.mask
land = ~ocean
vpd_tt = vpd_tt[land]


In [58]:
pearsonr(vpd_tt,prcp_tt)

(0.7173475084044314, 0.0)